In [2]:
from typing import TypedDict, List, Optional, Annotated, Set, Dict
import operator
import uuid

# For pretty printing the state (optional)
from langchain_core.runnables import RunnableLambda
from langgraph.graph import StateGraph, END

from IPython.display import Image, display


# Pub-Sub architecture

Okay, short and sweet:

1.  **Publishers:** Send messages (events/data) without knowing who receives them.
2.  **Subscribers:** Express interest in specific types of messages (topics/channels) without knowing the publishers.
3.  **Topics/Channels (or Broker):** An intermediary that filters and routes messages from publishers to interested subscribers.
4.  **Decoupling:** The key. Publishers and subscribers are independent and don't need direct knowledge of each other.

# v1 Failed attempt

Okay, let's implement a Pub-Sub (Publish-Subscriber) like architecture using LangGraph.

**Core Idea of Pub-Sub in this Context:**

In a traditional Pub-Sub system, a publisher sends messages to a topic, and multiple subscribers interested in that topic receive the message, often concurrently and independently.

With LangGraph, we can model this by:
1.  **Publisher Node:** A node that generates or processes some data (the "message").
2.  **State Update:** The publisher node updates the shared state with this message.
3.  **Subscriber Nodes:** Multiple subsequent nodes in the graph can then access this "published" message from the state and perform their own independent processing.

While LangGraph's default execution model is sequential (unless using `AsyncLangGraph` with `asyncio` for I/O-bound tasks or more complex configurations), we can still demonstrate the *flow* of information where one piece of data is made available to multiple consumers. The "subscribers" will process the message sequentially in this basic example, but they will all act upon the *same* published message.

**Key LangGraph Concepts Used:**

*   `StateGraph`: To define the graph.
*   `TypedDict`: To define the shared state.
*   `Annotated[List[str], operator.add]`: To allow nodes to append messages to lists in the state.
*   Nodes: Python functions that modify the state.
*   Edges: Define the sequence of node execution.
*   `END`: A special node to terminate the graph.

Let's create the Python code first, which can then be easily pasted into a Jupyter Notebook.



## Why is it failed?

It doesn't look like the pub-sub since we link a publisher and subscribers explicitly. There should not be explicit links between nodes. The subscribers should select messages not because the messages directed to them but because they are subscribed to some message parameters.

In [3]:
# --- 1. Define the State ---
# The state will hold the initial event, the published message,
# and logs from each subscriber.
class PubSubState(TypedDict):
    initial_event_id: str
    initial_event_payload: dict
    published_message: Optional[str]
    # We use Annotated with operator.add so that when a node returns a list
    # for these keys, it gets appended to the existing list in the state.
    subscriber_alpha_log: Annotated[List[str], operator.add]
    subscriber_beta_log: Annotated[List[str], operator.add]
    subscriber_critical_log: Annotated[List[str], operator.add]
    final_summary: Optional[str]

In [4]:
# --- 2. Define Node Functions ---

# Publisher Node
def publisher_node(state: PubSubState) -> dict:
    """
    Processes an initial event and "publishes" a message by updating the state.
    """
    print(f"\n--- Publisher Node (Event ID: {state['initial_event_id']}) ---")
    event_payload = state["initial_event_payload"]
    
    # Simulate processing the event
    message_content = f"Processed event: {event_payload.get('data', 'No data')}. Severity: {event_payload.get('severity', 'UNKNOWN')}"
    published_msg = f"[Published] {message_content}"
    print(f"Publishing: {published_msg}")
    
    return {
        "published_message": published_msg,
        # Initialize logs for subscribers if they are not present (though TypedDict usually handles this)
        "subscriber_alpha_log": [],
        "subscriber_beta_log": [],
        "subscriber_critical_log": []
    }

# Subscriber Node Alpha
def subscriber_alpha_node(state: PubSubState) -> dict:
    """
    Subscribes to the message and performs its action.
    """
    print("\n--- Subscriber Alpha Node ---")
    published_message = state["published_message"]
    if not published_message:
        print("No message published yet.")
        return {}
        
    log_entry = f"ALPHA: Received and logged: '{published_message}'"
    print(log_entry)
    return {"subscriber_alpha_log": [log_entry]} # Appends to the list

# Subscriber Node Beta
def subscriber_beta_node(state: PubSubState) -> dict:
    """
    Another subscriber that processes the message differently.
    """
    print("\n--- Subscriber Beta Node ---")
    published_message = state["published_message"]
    if not published_message:
        print("No message published yet.")
        return {}

    processed_info = f"BETA: Analyzed message: '{published_message}'. Action: Sent notification."
    print(processed_info)
    return {"subscriber_beta_log": [processed_info]}

# Conditional Subscriber Node (e.g., only for critical messages)
def subscriber_critical_node(state: PubSubState) -> dict:
    """
    Subscribes only if the message is deemed critical.
    """
    print("\n--- Subscriber Critical Node ---")
    published_message = state["published_message"]
    event_payload = state["initial_event_payload"]

    if not published_message:
        print("No message published yet.")
        return {}

    # Example condition: check severity from original payload
    if event_payload.get("severity") == "CRITICAL":
        log_entry = f"CRITICAL_HANDLER: ALERT! Critical event processed: '{published_message}'"
        print(log_entry)
        return {"subscriber_critical_log": [log_entry]}
    else:
        log_entry = f"CRITICAL_HANDLER: Message not critical, skipping: '{published_message}'"
        print(log_entry)
        return {"subscriber_critical_log": [log_entry]} # Log that it skipped


# Optional: A node to summarize or finalize
def summary_node(state: PubSubState) -> dict:
    print("\n--- Summary Node ---")
    # summary = f"  Event {state['initial_event_id']} processing complete.\n"
    # summary += f"  Published: {state['published_message']}\n"
    # summary += f"  Alpha Logs: {len(state['subscriber_alpha_log'])} entries\n"
    # summary += f"  Beta Logs: {len(state['subscriber_beta_log'])} entries\n"
    # summary += f"  Critical Logs: {len(state['subscriber_critical_log'])} entries"
    summary = f"""  Event {state['initial_event_id']} processing complete.
      Published: {state['published_message']}
      Alpha Logs: {len(state['subscriber_alpha_log'])} entries
      Beta Logs: {len(state['subscriber_beta_log'])} entries
      Critical Logs: {len(state['subscriber_critical_log'])} entries"""
    print(summary)
    return {"final_summary": summary}

In [5]:
# --- 3. Define the Graph Workflow ---
workflow = StateGraph(PubSubState)

# Add nodes to the graph
workflow.add_node("publisher", publisher_node)
workflow.add_node("subscriber_alpha", subscriber_alpha_node)
workflow.add_node("subscriber_beta", subscriber_beta_node)
workflow.add_node("subscriber_critical", subscriber_critical_node) # This node will always run
workflow.add_node("summary", summary_node)

# Set the entry point
workflow.set_entry_point("publisher")

# Define the edges - this creates a sequential flow for subscribers
# Publisher -> Subscriber Alpha -> Subscriber Beta -> Subscriber Critical -> Summary -> END
workflow.add_edge("publisher", "subscriber_alpha")
workflow.add_edge("subscriber_alpha", "subscriber_beta")
workflow.add_edge("subscriber_beta", "subscriber_critical")
workflow.add_edge("subscriber_critical", "summary")
workflow.add_edge("summary", END)

# --- 4. Compile the Graph ---
app = workflow.compile()

In [ ]:
display(Image(app.get_graph().draw_mermaid_png(max_retries=5, retry_delay=2.0)))


**Explanation:**

1.  **`PubSubState(TypedDict)`**:
    *   `initial_event_id`, `initial_event_payload`: Input to the system.
    *   `published_message`: This field will store the message produced by the `publisher_node`. All subsequent "subscriber" nodes will read from this.
    *   `subscriber_alpha_log`, `subscriber_beta_log`, `subscriber_critical_log`: These lists will store log entries from each subscriber. The `Annotated[List[str], operator.add]` tells LangGraph to append to these lists when a node returns a list for that key, rather than overwriting.
    *   `final_summary`: To store a summary at the end.

2.  **Node Functions (`publisher_node`, `subscriber_alpha_node`, etc.)**:
    *   Each function takes the current `state: PubSubState` as input.
    *   They perform their logic.
    *   They return a dictionary with keys corresponding to fields in `PubSubState` that they want to update.
    *   **Publisher (`publisher_node`)**: Simulates processing an event and sets the `published_message`.
    *   **Subscribers (`subscriber_alpha_node`, `subscriber_beta_node`, `subscriber_critical_node`)**: Read the `published_message` from the state and add their own logs. The `subscriber_critical_node` demonstrates conditional logic *within* a subscriber based on the event payload.

3.  **Graph Workflow (`StateGraph`)**:
    *   We add all our nodes.
    *   `set_entry_point("publisher")`: The graph starts with the publisher.
    *   `add_edge(...)`: We define a sequential flow: `publisher` -> `subscriber_alpha` -> `subscriber_beta` -> `subscriber_critical` -> `summary` -> `END`.
        *   This means after `publisher` finishes, `subscriber_alpha` runs. After `subscriber_alpha`, `subscriber_beta` runs, and so on.
        *   Crucially, `subscriber_alpha`, `subscriber_beta`, and `subscriber_critical` all have access to the `published_message` that was set by the `publisher_node` because the state is passed along and updated at each step.

4.  **Compilation and Execution**:
    *   `app = workflow.compile()`: Compiles the graph into a runnable application.
    *   `app.stream(input)`: Executes the graph with the given input and streams the output of each node as it executes.
    *   `app.invoke(input)`: Executes the graph and returns the final state.

**How this mimics Pub-Sub:**

*   **Publish:** The `publisher_node` "publishes" information by writing it to the shared `published_message` field in the state.
*   **Subscribe:** The `subscriber_alpha_node`, `subscriber_beta_node`, and `subscriber_critical_node` "subscribe" to this message by reading it from the state in their respective execution steps.
*   **Decoupling (Conceptual):** Although the subscribers run sequentially in this simple LangGraph setup, they are logically decoupled in terms of their processing. Subscriber Alpha doesn't need to know about Subscriber Beta's internals, only about the `published_message`.

**To make this a Jupyter Notebook:**

1.  Create a new Jupyter Notebook.
2.  In the first cell, install necessary packages if you haven't already:
    ```python
    !pip install langchain langgraph langchain_core
    ```
3.  Then, paste the Python code into subsequent cells. You can break it down:
    *   Cell 1: Imports
    *   Cell 2: State Definition (`PubSubState`)
    *   Cell 3: Node Functions (publisher, subscribers, summary)
    *   Cell 4: Graph Workflow Definition and Compilation
    *   Cell 5: Running the Graph (with normal and critical events) and printing outputs.

This structure provides a clear demonstration of how one node's output (the published message) can be consumed by multiple subsequent nodes, which is the essence of the Pub-Sub data flow pattern, adapted to LangGraph's stateful execution model. For true parallel execution of subscribers, you'd look into `AsyncLangGraph` and design your nodes as `async` functions performing I/O-bound operations.

In [17]:
# --- 5. Run the Graph ---

# Helper to pretty print state (optional)
def _print_state(state: dict):
    print("\n--- Current State ---")
    for k, v in state['values'].items():
        print(f"{k}: {v}")
    print("--------------------")
    return state['values']

# You can wrap app.stream with this for debugging
# app_with_state_print = app.stream | RunnableLambda(_print_state)


print("--- Running with a NORMAL event ---")
# state = PubSubState(
normal_event_id = str(uuid.uuid4())
normal_event_input = {
  "initial_event_id": normal_event_id,
  "initial_event_payload": {"data": "System nominal", "severity":   "NORMAL"}
}
final_state_normal = None

for s in app.stream(normal_event_input):
    # s is a dictionary where keys are node names and values are their outputs
    print(f"Output from node: {list(s.keys())[0]}") 
    final_state_normal = s[list(s.keys())[0]] # Get the state after the last node

print("\n\n=== Final State (NORMAL event) ===")
if final_state_normal: # final_state_normal will be the output of the last node (summary)
    # To get the full state, we'd typically inspect 's' in the loop
    # or invoke the graph and get the full final state directly if not streaming.
    # For simplicity, let's re-run with invoke to get the full final state.
    final_state_normal_full = app.invoke(normal_event_input)
    for key, value in final_state_normal_full.items():
        print(f"{key}: {value}")



--- Running with a NORMAL event ---

--- Publisher Node (Event ID: a928c610-7c1b-419c-87ff-1147f1251646) ---
Publishing: [Published] Processed event: System nominal. Severity: NORMAL
Output from node: publisher

--- Subscriber Alpha Node ---
ALPHA: Received and logged: '[Published] Processed event: System nominal. Severity: NORMAL'
Output from node: subscriber_alpha

--- Subscriber Beta Node ---
BETA: Analyzed message: '[Published] Processed event: System nominal. Severity: NORMAL'. Action: Sent notification.
Output from node: subscriber_beta

--- Subscriber Critical Node ---
CRITICAL_HANDLER: Message not critical, skipping: '[Published] Processed event: System nominal. Severity: NORMAL'
Output from node: subscriber_critical

--- Summary Node ---
  Event a928c610-7c1b-419c-87ff-1147f1251646 processing complete.
      Published: [Published] Processed event: System nominal. Severity: NORMAL
      Alpha Logs: 1 entries
      Beta Logs: 1 entries
      Critical Logs: 1 entries
Output from 

In [18]:
print("\n\n--- Running with a CRITICAL event ---")
critical_event_id = str(uuid.uuid4())
critical_event_input = {
    "initial_event_id": critical_event_id,
    "initial_event_payload": {"data": "System overload detected!", "severity": "CRITICAL"}
}
final_state_critical = None
for s in app.stream(critical_event_input):
    print(f"Output from node: {list(s.keys())[0]}")
    final_state_critical = s[list(s.keys())[0]]

print("\n\n--- Final State (CRITICAL event) ---")
if final_state_critical:
    final_state_critical_full = app.invoke(critical_event_input)
    for key, value in final_state_critical_full.items():
        print(f"{key}: {value}")



--- Running with a CRITICAL event ---

--- Publisher Node (Event ID: 90ea04b5-2005-4be7-a028-eb91f049e478) ---
Publishing: [Published] Processed event: System overload detected!. Severity: CRITICAL
Output from node: publisher

--- Subscriber Alpha Node ---
ALPHA: Received and logged: '[Published] Processed event: System overload detected!. Severity: CRITICAL'
Output from node: subscriber_alpha

--- Subscriber Beta Node ---
BETA: Analyzed message: '[Published] Processed event: System overload detected!. Severity: CRITICAL'. Action: Sent notification.
Output from node: subscriber_beta

--- Subscriber Critical Node ---
CRITICAL_HANDLER: ALERT! Critical event processed: '[Published] Processed event: System overload detected!. Severity: CRITICAL'
Output from node: subscriber_critical

--- Summary Node ---
  Event 90ea04b5-2005-4be7-a028-eb91f049e478 processing complete.
      Published: [Published] Processed event: System overload detected!. Severity: CRITICAL
      Alpha Logs: 1 entries


# v2 Multiple subscribers. Subscribers know anything about publisher.

My initial example was more of a sequential pipeline where data was passed along, not a true `Pub-Sub` model where subscribers react based on interest rather than direct wiring.

To better model `Pub-Sub` with `LangGraph`, we need:

- `Publisher`: A node that creates a message and "publishes" it with associated metadata (like topics or tags) into the shared state.
- `Router/Dispatcher`: A mechanism (a node or conditional edge logic) that inspects the published message's metadata.
- `Subscribers`: Nodes that have predefined "interests" (e.g., specific topics).
- `Dynamic Invocation`: The router/dispatcher determines which subscribers are interested in the current message and ensures only they are invoked. Subscribers are not explicitly chained after the publisher in a fixed sequence.


In [23]:
# --- 1. Define Subscriber Interests ---
# Each subscriber declares what topics it's interested in.
SUBSCRIBER_INTERESTS = {
    "subscriber_alpha": ["ALERT", "CRITICAL"],  # Alpha is interested in ALERT or CRITICAL messages
    "subscriber_beta": ["UPDATE", "GENERAL"],   # Beta is interested in UPDATE or GENERAL messages
    "subscriber_gamma": ["ALERT", "GENERAL", "STATS"], # Gamma is interested in ALERT, GENERAL or STATS
}

# --- 2. Define the State ---
class PubSubState(TypedDict):
    # Input for a single "publication" cycle
    event_id: str
    event_payload: dict  # e.g., {"data": "System overload", "type": "ALERT", "source": "sensor_123"}
    
    # State generated by the publisher for the current event
    published_message_content: Optional[str]
    published_message_topics: List[str]  # Topics/tags determined by the publisher
    
    # Control flow: Who should run for the current event?
    # List of subscriber node names that match the current event's topics
    subscribers_matched_for_event: List[str] 
    # Set of subscriber node names that have completed processing for the current event
    subscribers_completed_for_event: Set[str] 
    
    # Aggregated logs from subscribers
    # Using Annotated with operator.ior for dictionaries.
    # Each key is a subscriber name, value is a list of its log messages for the current event.
    subscriber_logs: Annotated[Dict[str, List[str]], operator.ior]
    
    # Could be used if processing a batch of events (not in this simple example)
    # current_event_index: int 
    # all_events_processed: bool


# --- Helper to initialize/reset logs for an event ---
def get_initial_subscriber_logs_for_event() -> Dict[str, List[str]]:
    return {sub_name: [] for sub_name in SUBSCRIBER_INTERESTS.keys()}


In [24]:
# --- 3. Define Node Functions ---

def publisher_node(state: PubSubState) -> Dict:
    """
    Processes an incoming event, "publishes" a message with derived topics,
    and resets subscriber tracking for this new event.
    """
    event_id = state["event_id"]
    payload = state["event_payload"]
    print(f"\n--- Publisher Node ---")
    print(f"Processing Event ID: {event_id}, Payload: {payload}")
    
    # Simulate deriving message content and topics from the payload
    message_content = f"Event: {payload.get('data', 'N/A')} (Source: {payload.get('source', 'Unknown')})"
    
    # Determine topics for the message (example logic)
    derived_topics = []
    event_type = payload.get("type", "GENERAL").upper()
    if event_type:
        derived_topics.append(event_type)
    
    if "critical" in payload.get("data", "").lower() and "CRITICAL" not in derived_topics:
        derived_topics.append("CRITICAL")
    if "stats" in payload.get("data", "").lower() and "STATS" not in derived_topics:
        derived_topics.append("STATS")
    if not derived_topics: # Ensure there's always at least one topic
        derived_topics.append("GENERAL")

    print(f"Published Message: '{message_content}' with Topics: {derived_topics}")
    
    return {
        "published_message_content": message_content,
        "published_message_topics": derived_topics,
        "subscribers_matched_for_event": [],  # Reset for this new event
        "subscribers_completed_for_event": set(),  # Reset for this new event
        "subscriber_logs": get_initial_subscriber_logs_for_event() # Reset logs for the new event
    }

def router_node(state: PubSubState) -> Dict:
    """
    Inspects the published message's topics and determines which subscribers
    are interested (i.e., subscribed to at least one of those topics).
    """
    print("\n--- Router Node ---")
    published_topics = state["published_message_topics"]
    matched_subscribers = []
    
    for sub_name, interests in SUBSCRIBER_INTERESTS.items():
        # Check if any of the subscriber's interests (topics they care about)
        # are present in the message's topics
        if any(topic in published_topics for topic in interests):
            matched_subscribers.append(sub_name)
            
    print(f"Message Topics: {published_topics}")
    print(f"Subscribers Matched for these topics: {matched_subscribers}")
    
    return {"subscribers_matched_for_event": matched_subscribers}


# --- Generic Subscriber Node Factory ---
# This creates a unique node function for each named subscriber.
def create_subscriber_node(subscriber_name: str):
    def subscriber_node_logic(state: PubSubState) -> Dict:
        # This node should only really execute if it's its turn.
        # The conditional routing should handle this, but defensive checks are fine.
        print(f"\n--- {subscriber_name.upper()} Node ---")

        message = state["published_message_content"]
        topics = state["published_message_topics"]
        
        # Simulate subscriber processing the message
        log_entry = (f"{subscriber_name.upper()}: Processed message '{message}' "
                     f"(Relevant Topics for this message: {topics})")
        print(log_entry)
        
        # Update logs:
        # Read current logs for this specific subscriber, append new entry.
        # The state's `subscriber_logs` is Annotated with operator.ior,
        # so we return a dict that will be merged.
        new_log_for_this_subscriber = {
            subscriber_name: state["subscriber_logs"].get(subscriber_name, []) + [log_entry]
        }
        
        # Mark this subscriber as having completed for the current event
        updated_completed_set = set(state["subscribers_completed_for_event"]) # Create a mutable copy
        updated_completed_set.add(subscriber_name)
        
        return {
            "subscriber_logs": new_log_for_this_subscriber, # This dict will be merged into the state
            "subscribers_completed_for_event": updated_completed_set
        }
    return subscriber_node_logic

# Create actual subscriber node functions
subscriber_alpha_node = create_subscriber_node("subscriber_alpha")
subscriber_beta_node = create_subscriber_node("subscriber_beta")
subscriber_gamma_node = create_subscriber_node("subscriber_gamma")


def final_summary_node(state: PubSubState) -> Dict:
    """
    Prints a summary of the processing for the current event.
    """
    print("\n--- Event Processing Summary ---")
    print(f"Event ID: {state['event_id']}")
    print(f"Published Message: {state['published_message_content']}")
    print(f"Message Topics: {state['published_message_topics']}")
    print(f"Matched Subscribers: {state['subscribers_matched_for_event']}")
    print(f"Completed Subscribers: {state['subscribers_completed_for_event']}")
    print("Subscriber Logs for this event:")
    
    # Sort by subscriber name for consistent output
    sorted_log_keys = sorted(state["subscriber_logs"].keys())

    for sub_name in sorted_log_keys:
        logs = state["subscriber_logs"][sub_name]
        if logs:  # Only print if the subscriber actually logged something for this event
            print(f"  Logs from {sub_name.upper()}:")
            for log_entry in logs:
                # Extract just the processing part of the log for brevity
                processed_part = log_entry.split(': ', 1)[1] if ': ' in log_entry else log_entry
                print(f"    - {processed_part}")
        # else:
            # print(f"  No logs from {sub_name.upper()} for this event.") # Optional: for debugging
    print("---------------------------------")
    return {} # No state change, just a terminal action for this event's processing


In [25]:
# --- 4. Conditional Logic for Dispatching Subscribers ---
def route_to_next_subscriber_or_summary(state: PubSubState) -> str:
    """
    This function is the core of the Pub-Sub dispatch logic.
    It decides which node to go to next after the router_node has identified
    matched subscribers, or after a subscriber has finished.

    Returns:
        The name of the next node to execute.
    """
    matched_subscribers = state["subscribers_matched_for_event"]
    completed_subscribers = state["subscribers_completed_for_event"]
    
    if not matched_subscribers:
        # No subscribers were interested in this event's topics.
        print("Conditional Dispatch: No subscribers matched this event. Routing to summary.")
        return "summary"

    # Find the next matched subscriber that hasn't completed yet
    for sub_name in matched_subscribers:
        if sub_name not in completed_subscribers:
            print(f"Conditional Dispatch: Routing to next pending subscriber: {sub_name}")
            return sub_name  # Return the node name of the subscriber to run
            
    # If we reach here, all matched subscribers for the current event have completed.
    print("Conditional Dispatch: All matched subscribers have processed the event. Routing to summary.")
    return "summary"


In [36]:
# --- 5. Define the Graph Workflow ---
workflow = StateGraph(PubSubState)

# Add nodes to the graph
workflow.add_node("publisher", publisher_node)
workflow.add_node("router", router_node)

# Add each defined subscriber node to the graph
workflow.add_node("subscriber_alpha", subscriber_alpha_node)
workflow.add_node("subscriber_beta", subscriber_beta_node)
workflow.add_node("subscriber_gamma", subscriber_gamma_node)

workflow.add_node("summary", final_summary_node)


# --- Define Edges and Control Flow ---

# Start with the publisher
workflow.set_entry_point("publisher")

# After publishing, route to determine interested subscribers
workflow.add_edge("publisher", "router")

# This is the main dispatch hub:
# After the router identifies potential subscribers, or after a subscriber finishes,
# this conditional edge decides what to do next.
workflow.add_conditional_edges(
    "router",  # The decision is made *after* the router_node runs.
    route_to_next_subscriber_or_summary,  # The function that returns the name of the next node.
    {
        # Map returned names to actual graph nodes
        "subscriber_alpha": "subscriber_alpha",
        "subscriber_beta": "subscriber_beta",
        "subscriber_gamma": "subscriber_gamma",
        "summary": "summary"  # If no more subscribers for this event, or none matched
    }
)

# After each subscriber finishes, it needs to go back to the decision point
# to allow the next pending subscriber (for the same event) to run, or to go to summary.
# The 'router' node here serves as that decision point trigger because the conditional
# edge is attached to it. When a subscriber transitions to 'router', the router logic
# runs (it will find the same matched_subscribers for the current event), and then
# `route_to_next_subscriber_or_summary` is called again, which will now see the updated
# `subscribers_completed_for_event` and pick the next one or go to summary.
workflow.add_edge("subscriber_alpha", "router")
workflow.add_edge("subscriber_beta", "router")
workflow.add_edge("subscriber_gamma", "router")

# Once the summary node is reached for an event, the graph ends for that event.
workflow.add_edge("summary", END)


# --- 6. Compile the Graph ---
# mem_saver = MemorySaver() # Optional: for inspecting state at each step if needed
app = workflow.compile() # checkpoint_saver=mem_saver if using checkpoints



In [37]:
from IPython.display import Image, display

display(Image(app.get_graph().draw_mermaid_png()))

ValueError: Failed to reach https://mermaid.ink/ API while trying to render your graph after 1 retries. To resolve this issue:
1. Check your internet connection and try again
2. Try with higher retry settings: `draw_mermaid_png(..., max_retries=5, retry_delay=2.0)`
3. Use the Pyppeteer rendering method which will render your graph locally in a browser: `draw_mermaid_png(..., draw_method=MermaidDrawMethod.PYPPETEER)`

In [27]:
# --- 7. Run the Graph with Different Example Events ---

def run_event_through_pubsub(event_payload: dict):
    event_id = str(uuid.uuid4())
    print(f"\n\n<<<<< STARTING NEW EVENT: ID {event_id}, Type: {event_payload.get('type', 'N/A')} >>>>>")
    
    initial_state = {
        "event_id": event_id,
        "event_payload": event_payload,
        # Other fields will be initialized by the publisher or are optional initially
    }
    
    # Stream the execution to see node outputs
    for step_output in app.stream(initial_state):
        node_name = list(step_output.keys())[0]
        print(f"Output from node: {node_name} (Value keys: {list(step_output[node_name].keys()) if step_output[node_name] else 'None'})")
        # For very verbose output of state after each node:
        # print(f"Full state after {node_name}: {step_output[node_name]}")
    
    print(f"<<<<< FINISHED EVENT: ID {event_id} >>>>>")

In [28]:
# Example Event 1: An ALERT that should trigger Alpha and Gamma
event1_payload = {"data": "System critical failure imminent!", "type": "ALERT", "source": "CPU_Monitor"}
run_event_through_pubsub(event1_payload)



<<<<< STARTING NEW EVENT: ID d544f6a9-932b-47df-8bda-d844b4c2877f, Type: ALERT >>>>>

--- Publisher Node ---
Processing Event ID: d544f6a9-932b-47df-8bda-d844b4c2877f, Payload: {'data': 'System critical failure imminent!', 'type': 'ALERT', 'source': 'CPU_Monitor'}
Published Message: 'Event: System critical failure imminent! (Source: CPU_Monitor)' with Topics: ['ALERT', 'CRITICAL']
Output from node: publisher (Value keys: ['published_message_content', 'published_message_topics', 'subscribers_matched_for_event', 'subscribers_completed_for_event', 'subscriber_logs'])

--- Router Node ---
Message Topics: ['ALERT', 'CRITICAL']
Subscribers Matched for these topics: ['subscriber_alpha', 'subscriber_gamma']
Conditional Dispatch: Routing to next pending subscriber: subscriber_alpha
Output from node: router (Value keys: ['subscribers_matched_for_event'])

--- SUBSCRIBER_ALPHA Node ---
SUBSCRIBER_ALPHA: Processed message 'Event: System critical failure imminent! (Source: CPU_Monitor)' (Relevant

In [29]:
# Example Event 2: An UPDATE that should trigger Beta
event2_payload = {"data": "User profile for 'jane_doe' updated.", "type": "UPDATE", "source": "AuthService"}
run_event_through_pubsub(event2_payload)




<<<<< STARTING NEW EVENT: ID aa48ed2e-ea71-4b35-b58a-83200ecf3e8b, Type: UPDATE >>>>>

--- Publisher Node ---
Processing Event ID: aa48ed2e-ea71-4b35-b58a-83200ecf3e8b, Payload: {'data': "User profile for 'jane_doe' updated.", 'type': 'UPDATE', 'source': 'AuthService'}
Published Message: 'Event: User profile for 'jane_doe' updated. (Source: AuthService)' with Topics: ['UPDATE']
Output from node: publisher (Value keys: ['published_message_content', 'published_message_topics', 'subscribers_matched_for_event', 'subscribers_completed_for_event', 'subscriber_logs'])

--- Router Node ---
Message Topics: ['UPDATE']
Subscribers Matched for these topics: ['subscriber_beta']
Conditional Dispatch: Routing to next pending subscriber: subscriber_beta
Output from node: router (Value keys: ['subscribers_matched_for_event'])

--- SUBSCRIBER_BETA Node ---
SUBSCRIBER_BETA: Processed message 'Event: User profile for 'jane_doe' updated. (Source: AuthService)' (Relevant Topics for this message: ['UPDATE'

In [30]:
# Example Event 3: A GENERAL STATS event that should trigger Beta and Gamma
event3_payload = {"data": "Hourly user activity stats generated.", "type": "GENERAL", "source": "AnalyticsEngine"}
run_event_through_pubsub(event3_payload) # Note: "STATS" might also be derived by publisher




<<<<< STARTING NEW EVENT: ID 1732b3a5-af92-45cb-8918-cd51dd95b124, Type: GENERAL >>>>>

--- Publisher Node ---
Processing Event ID: 1732b3a5-af92-45cb-8918-cd51dd95b124, Payload: {'data': 'Hourly user activity stats generated.', 'type': 'GENERAL', 'source': 'AnalyticsEngine'}
Published Message: 'Event: Hourly user activity stats generated. (Source: AnalyticsEngine)' with Topics: ['GENERAL', 'STATS']
Output from node: publisher (Value keys: ['published_message_content', 'published_message_topics', 'subscribers_matched_for_event', 'subscribers_completed_for_event', 'subscriber_logs'])

--- Router Node ---
Message Topics: ['GENERAL', 'STATS']
Subscribers Matched for these topics: ['subscriber_beta', 'subscriber_gamma']
Conditional Dispatch: Routing to next pending subscriber: subscriber_beta
Output from node: router (Value keys: ['subscribers_matched_for_event'])

--- SUBSCRIBER_BETA Node ---
SUBSCRIBER_BETA: Processed message 'Event: Hourly user activity stats generated. (Source: Analy

In [31]:
# Example Event 4: An event with a type no one is directly subscribed to (becomes GENERAL)
event4_payload = {"data": "Routine maintenance log.", "type": "LOG", "source": "SystemDaemon"}
run_event_through_pubsub(event4_payload) # Publisher might default this to "GENERAL" if "LOG" is not a primary topic.



<<<<< STARTING NEW EVENT: ID 465c885c-a3a6-4195-b6f1-3f581fdd78e7, Type: LOG >>>>>

--- Publisher Node ---
Processing Event ID: 465c885c-a3a6-4195-b6f1-3f581fdd78e7, Payload: {'data': 'Routine maintenance log.', 'type': 'LOG', 'source': 'SystemDaemon'}
Published Message: 'Event: Routine maintenance log. (Source: SystemDaemon)' with Topics: ['LOG']
Output from node: publisher (Value keys: ['published_message_content', 'published_message_topics', 'subscribers_matched_for_event', 'subscribers_completed_for_event', 'subscriber_logs'])

--- Router Node ---
Message Topics: ['LOG']
Subscribers Matched for these topics: []
Conditional Dispatch: No subscribers matched this event. Routing to summary.
Output from node: router (Value keys: ['subscribers_matched_for_event'])

--- Event Processing Summary ---
Event ID: 465c885c-a3a6-4195-b6f1-3f581fdd78e7
Published Message: Event: Routine maintenance log. (Source: SystemDaemon)
Message Topics: ['LOG']
Matched Subscribers: []
Completed Subscribers:

In [32]:
# Example Event 5: An event with CRITICAL and STATS implications
event5_payload = {"data": "Critical spike in error stats detected!", "type": "ALERT", "source": "Aggregator"}
run_event_through_pubsub(event5_payload) # Alpha (ALERT, CRITICAL) and Gamma (ALERT, STATS)



<<<<< STARTING NEW EVENT: ID 6a33678c-0e3c-44a5-973f-b1f1cf832f6c, Type: ALERT >>>>>

--- Publisher Node ---
Processing Event ID: 6a33678c-0e3c-44a5-973f-b1f1cf832f6c, Payload: {'data': 'Critical spike in error stats detected!', 'type': 'ALERT', 'source': 'Aggregator'}
Published Message: 'Event: Critical spike in error stats detected! (Source: Aggregator)' with Topics: ['ALERT', 'CRITICAL', 'STATS']
Output from node: publisher (Value keys: ['published_message_content', 'published_message_topics', 'subscribers_matched_for_event', 'subscribers_completed_for_event', 'subscriber_logs'])

--- Router Node ---
Message Topics: ['ALERT', 'CRITICAL', 'STATS']
Subscribers Matched for these topics: ['subscriber_alpha', 'subscriber_gamma']
Conditional Dispatch: Routing to next pending subscriber: subscriber_alpha
Output from node: router (Value keys: ['subscribers_matched_for_event'])

--- SUBSCRIBER_ALPHA Node ---
SUBSCRIBER_ALPHA: Processed message 'Event: Critical spike in error stats detected

# v3 + Multiple Publishers


The Subscriber logic is the same.

Okay, let's refactor the code to include multiple distinct publisher nodes. The core idea is:

1.  **Identify Event Source:** The incoming event will need some way to indicate its origin or type (e.g., "sensor_event", "user_action_event", "system_event").
2.  **Publisher Router Node:** An initial node in the graph will look at this event source and route it to the appropriate specialized publisher node.
3.  **Specialized Publisher Nodes:** We'll create a few different publisher functions, each tailored to a type of event. For example:
    *   `publisher_sensor_events_node`: Handles events from IoT sensors.
    *   `publisher_user_actions_node`: Handles events triggered by user interactions.
    *   `publisher_system_alerts_node`: Handles internal system alerts.
4.  **Common Subscriber Router:** After a specialized publisher node processes the event and "publishes" its message (i.e., sets `published_message_content` and `published_message_topics` in the state), the flow goes to the *same* `subscriber_router_node` we had before. This node doesn't care *which* publisher created the message, only what its topics are.
5.  **Subscriber Logic:** The subscriber matching and execution logic remains the same.


This structure now clearly separates different publishing concerns while maintaining a common pipeline for subscriber matching and execution, which is much closer to how multiple publishers would interact with a Pub-Sub system.

In [49]:
# pub_sub_langgraph_v3_multiple_publishers.py

from typing import TypedDict, List, Optional, Annotated, Set, Dict, Any
import operator
import uuid

from langgraph.graph import StateGraph, END

# --- 1. Define Subscriber Interests ---
SUBSCRIBER_INTERESTS = {
    "subscriber_alpha": ["ALERT", "CRITICAL_SENSOR"],
    "subscriber_beta": ["UPDATE", "USER_ACTION", "GENERAL"],
    "subscriber_gamma": ["ALERT", "SYSTEM_HEALTH", "STATS", "MAINTENANCE"],
    "subscriber_audit": ["USER_ACTION", "CRITICAL_SENSOR", "SYSTEM_CONFIG_CHANGE", "SECURITY_ALERT"]
}

# --- 2. Define the State ---
class PubSubState(TypedDict):
    event_id: str
    # initial_event_payload will now contain a 'source_type' key
    initial_event_payload: Dict[str, Any] 
    
    # Info about which publisher handled this event (for clarity/logging)
    publisher_identity: Optional[str]
    
    # Standard fields for the published message
    published_message_content: Optional[str]
    published_message_topics: List[str]
    
    # Subscriber control flow
    subscribers_matched_for_event: List[str] 
    subscribers_completed_for_event: Set[str] 
    
    subscriber_logs: Annotated[Dict[str, List[str]], operator.ior]


# --- Helper to initialize/reset logs for an event ---
def get_initial_subscriber_logs_for_event() -> Dict[str, List[str]]:
    return {sub_name: [] for sub_name in SUBSCRIBER_INTERESTS.keys()}

# --- Utility to reset parts of state for a new event publication ---
# def reset_publication_state_fields() -> Dict: ## ERROR
#     return {
#         "published_message_content": None,
#         "published_message_topics": [],
#         "subscribers_matched_for_event": [],
#         "subscribers_completed_for_event": set(),
#         "subscriber_logs": get_initial_subscriber_logs_for_event()
#     }

def reset_publication_state_fields() -> Dict:
    """
    Resets state fields that are specific to the subscriber processing loop
    for a new event/publication.
    """
    return {
        # "published_message_content": None, # DO NOT RESET THESE HERE
        # "published_message_topics": [],    # PUBLISHER SETS THEM
        "subscribers_matched_for_event": [],  # Reset for the new message
        "subscribers_completed_for_event": set(), # Reset for the new message
        "subscriber_logs": get_initial_subscriber_logs_for_event() # Reset for the new message
    }

In [50]:
# --- 3. Define Specialized Publisher Node Functions ---

def publisher_sensor_events_node(state: PubSubState) -> Dict:
    event_id = state["event_id"]
    payload = state["initial_event_payload"].get("details", {})
    publisher_id = "SensorPublisher"
    print(f"\n--- Publisher: {publisher_id} (Event ID: {event_id}) ---")
    
    reading = payload.get("reading", "N/A")
    unit = payload.get("unit", "")
    status = payload.get("status", "NORMAL").upper()

    message_content = f"Sensor Event: ID {payload.get('sensor_id', 'UnknownSensor')}, Reading {reading}{unit}, Status: {status}."
    derived_topics = ["SENSOR_DATA"]
    if status == "CRITICAL":
        derived_topics.extend(["CRITICAL_SENSOR", "ALERT"])
    elif status == "WARNING":
        derived_topics.extend(["WARNING_SENSOR", "ALERT"])
    
    print(f"Publishing from {publisher_id}: '{message_content}' with Topics: {derived_topics}")
    
    return {
        "publisher_identity": publisher_id,
        "published_message_content": message_content,
        "published_message_topics": derived_topics,
        **reset_publication_state_fields() # Reset subscriber tracking for this new message
    }

def publisher_user_actions_node(state: PubSubState) -> Dict:
    event_id = state["event_id"]
    payload = state["initial_event_payload"].get("details", {})
    publisher_id = "UserActionPublisher"
    print(f"\n--- Publisher: {publisher_id} (Event ID: {event_id}) ---")

    user_id = payload.get("user_id", "anonymous")
    action = payload.get("action", "unknown_action").upper()
    resource = payload.get("resource", "N/A")

    message_content = f"User Action: User '{user_id}' performed '{action}' on '{resource}'."
    derived_topics = ["USER_ACTION", action] # e.g., USER_ACTION, LOGIN

    if "DELETE" in action or "SUSPEND" in action:
        derived_topics.extend(["CRITICAL_USER_ACTION", "ALERT", "SECURITY_ALERT"])
    if action == "CONFIG_CHANGE" and resource.startswith("system"): # Example
        derived_topics.append("SYSTEM_CONFIG_CHANGE")

    print(f"Publishing from {publisher_id}: '{message_content}' with Topics: {derived_topics}")
    
    return {
        "publisher_identity": publisher_id,
        "published_message_content": message_content,
        "published_message_topics": derived_topics,
        **reset_publication_state_fields()
    }

def publisher_system_events_node(state: PubSubState) -> Dict:
    event_id = state["event_id"]
    payload = state["initial_event_payload"].get("details", {})
    publisher_id = "SystemEventPublisher"
    print(f"\n--- Publisher: {publisher_id} (Event ID: {event_id}) ---")

    event_name = payload.get("event_name", "generic_system_event").upper()
    severity = payload.get("severity", "INFO").upper()
    component = payload.get("component", "CORE").upper()

    message_content = f"System Event: '{event_name}' from component '{component}', Severity: {severity}."
    derived_topics = ["SYSTEM_EVENT", f"SYSTEM_{component}", f"SEVERITY_{severity}"]

    if severity in ["ERROR", "FATAL", "CRITICAL"]:
        derived_topics.append("ALERT")
    if event_name == "HEALTH_CHECK_FAIL":
        derived_topics.append("SYSTEM_HEALTH")
    if "MAINTENANCE" in event_name:
        derived_topics.append("MAINTENANCE")
    if "STATS_REPORT" in event_name:
        derived_topics.append("STATS")

    print(f"Publishing from {publisher_id}: '{message_content}' with Topics: {derived_topics}")
    
    return {
        "publisher_identity": publisher_id,
        "published_message_content": message_content,
        "published_message_topics": derived_topics,
        **reset_publication_state_fields()
    }

In [51]:
# --- 4. Publisher Router Node ---
def route_to_correct_publisher(state: PubSubState) -> str:
    """
    Inspects the initial event payload to determine which publisher logic to use.
    The `initial_event_payload` must contain a `source_type` field.
    """
    print("\n--- Event Source Router (Selecting Publisher) ---")
    source_type = state["initial_event_payload"].get("source_type", "UNKNOWN").upper()
    
    if source_type == "SENSOR_EVENT":
        print("Routing to SENSOR event publisher.")
        return "publisher_sensor"
    elif source_type == "USER_ACTION_EVENT":
        print("Routing to USER_ACTION event publisher.")
        return "publisher_user_action"
    elif source_type == "SYSTEM_INTERNAL_EVENT":
        print("Routing to SYSTEM event publisher.")
        return "publisher_system"
    else:
        print(f"Warning: Unknown source_type '{source_type}'. Defaulting to system publisher.")
        # Fallback or error handling: could route to a generic/error publisher
        return "publisher_system" 

In [52]:
# --- 5. Subscriber Logic (Router and Individual Subscribers) ---
# This part remains largely the same as before.

def subscriber_router_node(state: PubSubState) -> Dict:
    print("\n--- Subscriber Router Node (Matching Subscribers to Message) ---")
    if not state.get("published_message_topics"): # Should always be set by a publisher
        print("Error: No message topics found in state for subscriber routing.")
        return {"subscribers_matched_for_event": []}

    published_topics = state["published_message_topics"]
    matched_subscribers = []
    for sub_name, interests in SUBSCRIBER_INTERESTS.items():
        if any(topic in published_topics for topic in interests):
            matched_subscribers.append(sub_name)
            
    print(f"Message Topics: {published_topics}")
    print(f"Subscribers Matched for these topics: {matched_subscribers}")
    return {"subscribers_matched_for_event": matched_subscribers}

def create_subscriber_node(subscriber_name: str): # Factory remains the same
    def subscriber_node_logic(state: PubSubState) -> Dict:
        print(f"\n--- {subscriber_name.upper()} Node ---")
        message = state["published_message_content"]
        topics = state["published_message_topics"]
        
        log_entry = (f"{subscriber_name.upper()}: Processed message '{message}' "
                     f"(Originated from: {state.get('publisher_identity', 'Unknown Publisher')}, Topics: {topics})")
        print(log_entry)
        
        current_logs_for_subscriber = state["subscriber_logs"].get(subscriber_name, [])
        new_log_for_this_subscriber = {
            subscriber_name: current_logs_for_subscriber + [log_entry]
        }
        
        updated_completed_set = set(state.get("subscribers_completed_for_event", set()))
        updated_completed_set.add(subscriber_name)
        
        return {
            "subscriber_logs": new_log_for_this_subscriber, # Merge this dict into state
            "subscribers_completed_for_event": updated_completed_set
        }
    return subscriber_node_logic

subscriber_alpha_node = create_subscriber_node("subscriber_alpha")
subscriber_beta_node = create_subscriber_node("subscriber_beta")
subscriber_gamma_node = create_subscriber_node("subscriber_gamma")
subscriber_audit_node = create_subscriber_node("subscriber_audit")

def final_summary_node(state: PubSubState) -> Dict: # Remains the same
    print("\n--- Event Processing Summary ---")
    print(f"Event ID: {state['event_id']}")
    print(f"Processed by Publisher: {state.get('publisher_identity', 'N/A')}")
    print(f"Published Message: {state.get('published_message_content', 'N/A')}")
    print(f"Message Topics: {state.get('published_message_topics', [])}")
    print(f"Matched Subscribers: {state.get('subscribers_matched_for_event', [])}")
    print(f"Completed Subscribers: {state.get('subscribers_completed_for_event', set())}")
    print("Subscriber Logs for this event:")
    
    sorted_log_keys = sorted(state.get("subscriber_logs", {}).keys())
    for sub_name in sorted_log_keys:
        logs = state["subscriber_logs"].get(sub_name, [])
        if logs:
            print(f"  Logs from {sub_name.upper()}:")
            for log_entry in logs:
                processed_part = log_entry.split(': ', 1)[1] if ': ' in log_entry else log_entry
                print(f"    - {processed_part}")
    print("---------------------------------")
    return {}

def route_to_next_subscriber_or_summary(state: PubSubState) -> str: # Remains the same
    matched_subscribers = state.get("subscribers_matched_for_event", [])
    completed_subscribers = state.get("subscribers_completed_for_event", set())
    
    if not matched_subscribers:
        print("Conditional Subscriber Dispatch: No subscribers matched. Routing to summary.")
        return "summary"

    for sub_name in matched_subscribers:
        if sub_name not in completed_subscribers:
            print(f"Conditional Subscriber Dispatch: Routing to next pending subscriber: {sub_name}")
            return sub_name  
            
    print("Conditional Subscriber Dispatch: All matched subscribers processed. Routing to summary.")
    return "summary"

In [53]:
# --- 6. Define the Graph Workflow ---
workflow = StateGraph(PubSubState)

# Add an initial node that doesn't do much, just to kick off the publisher routing
# This node could also be used for common pre-processing if needed.
def initial_event_intake_node(state: PubSubState) -> Dict:
    print("\n--- Initial Event Intake ---")
    # Can set event_id here if not passed in initial state, or other common initializations
    # For now, it's mainly a starting point for the conditional publisher routing.
    return {"publisher_identity": None} # Initialize to ensure key exists

workflow.add_node("initial_intake", initial_event_intake_node)

# Add specialized publisher nodes
workflow.add_node("publisher_sensor", publisher_sensor_events_node)
workflow.add_node("publisher_user_action", publisher_user_actions_node)
workflow.add_node("publisher_system", publisher_system_events_node)

# Add subscriber-side nodes (router, individual subscribers, summary)
workflow.add_node("subscriber_router", subscriber_router_node)
workflow.add_node("subscriber_alpha", subscriber_alpha_node)
workflow.add_node("subscriber_beta", subscriber_beta_node)
workflow.add_node("subscriber_gamma", subscriber_gamma_node)
workflow.add_node("subscriber_audit", subscriber_audit_node)
workflow.add_node("summary", final_summary_node)

# --- Define Edges and Control Flow ---
workflow.set_entry_point("initial_intake")

# Conditional Edges from Initial Intake to specific Publisher Nodes
workflow.add_conditional_edges(
    "initial_intake",
    route_to_correct_publisher, # This function returns the name of the next node
    { # Map of returned names to actual graph nodes
        "publisher_sensor": "publisher_sensor",
        "publisher_user_action": "publisher_user_action",
        "publisher_system": "publisher_system",
    }
)

# ALL specialized publishers, after doing their work, route to the COMMON subscriber_router
workflow.add_edge("publisher_sensor", "subscriber_router")
workflow.add_edge("publisher_user_action", "subscriber_router")
workflow.add_edge("publisher_system", "subscriber_router")

# Conditional Edges from Subscriber Router to specific Subscriber Nodes (same as before)
workflow.add_conditional_edges(
    "subscriber_router",
    route_to_next_subscriber_or_summary,
    {
        "subscriber_alpha": "subscriber_alpha",
        "subscriber_beta": "subscriber_beta",
        "subscriber_gamma": "subscriber_gamma",
        "subscriber_audit": "subscriber_audit",
        "summary": "summary"
    }
)

# After each subscriber finishes, loop back to subscriber_router (same as before)
workflow.add_edge("subscriber_alpha", "subscriber_router")
workflow.add_edge("subscriber_beta", "subscriber_router")
workflow.add_edge("subscriber_gamma", "subscriber_router")
workflow.add_edge("subscriber_audit", "subscriber_router")

workflow.add_edge("summary", END)

# --- 7. Compile the Graph ---
app = workflow.compile()

In [54]:
# --- 8. Run the Graph with Different Example Events ---
def run_event_through_pubsub(event_payload_with_source: dict):
    event_id = str(uuid.uuid4()) # Generate a unique ID for each event run
    print(f"\n\n<<<<< STARTING NEW EVENT: ID {event_id}, Source Type: {event_payload_with_source.get('source_type', 'N/A')} >>>>>")
    
    initial_state_for_run = {
        "event_id": event_id,
        "initial_event_payload": event_payload_with_source,
        # Other state fields will be populated by the graph nodes
    }
    
    final_node_output = None
    for step_output in app.stream(initial_state_for_run):
        node_name = list(step_output.keys())[0]
        # print(f"Debug: Output from node: {node_name}") # Optional: for debugging flow
        final_node_output = step_output[node_name] # Keep track of the last state snapshot
    
    # print(f"\n--- Final State for Event {event_id} (after {node_name}) ---")
    # for key, value in final_node_output.items():
    # print(f"  {key}: {value}") # Can be verbose, summary node already prints key info
    print(f"<<<<< FINISHED EVENT: ID {event_id} >>>>>")


In [55]:
# Example Events:
event_sensor_critical = {
    "source_type": "SENSOR_EVENT",
    "details": {"sensor_id": "temp_CPU0", "reading": 95.5, "unit": "C", "status": "CRITICAL"}
}
run_event_through_pubsub(event_sensor_critical) # Expect Alpha, Audit



<<<<< STARTING NEW EVENT: ID 42237c01-d67f-4880-a4a3-1c537496a5c3, Source Type: SENSOR_EVENT >>>>>

--- Initial Event Intake ---

--- Event Source Router (Selecting Publisher) ---
Routing to SENSOR event publisher.

--- Publisher: SensorPublisher (Event ID: 42237c01-d67f-4880-a4a3-1c537496a5c3) ---
Publishing from SensorPublisher: 'Sensor Event: ID temp_CPU0, Reading 95.5C, Status: CRITICAL.' with Topics: ['SENSOR_DATA', 'CRITICAL_SENSOR', 'ALERT']

--- Subscriber Router Node (Matching Subscribers to Message) ---
Message Topics: ['SENSOR_DATA', 'CRITICAL_SENSOR', 'ALERT']
Subscribers Matched for these topics: ['subscriber_alpha', 'subscriber_gamma', 'subscriber_audit']
Conditional Subscriber Dispatch: Routing to next pending subscriber: subscriber_alpha

--- SUBSCRIBER_ALPHA Node ---
SUBSCRIBER_ALPHA: Processed message 'Sensor Event: ID temp_CPU0, Reading 95.5C, Status: CRITICAL.' (Originated from: SensorPublisher, Topics: ['SENSOR_DATA', 'CRITICAL_SENSOR', 'ALERT'])

--- Subscriber 

In [56]:
event_user_delete = {
    "source_type": "USER_ACTION_EVENT",
    "details": {"user_id": "operator01", "action": "DELETE_CRITICAL_RESOURCE", "resource": "db_master_table"}
}
run_event_through_pubsub(event_user_delete) # Expect Beta, Audit



<<<<< STARTING NEW EVENT: ID 5228f7fd-ce96-4939-b0d4-25de4f012dd3, Source Type: USER_ACTION_EVENT >>>>>

--- Initial Event Intake ---

--- Event Source Router (Selecting Publisher) ---
Routing to USER_ACTION event publisher.

--- Publisher: UserActionPublisher (Event ID: 5228f7fd-ce96-4939-b0d4-25de4f012dd3) ---
Publishing from UserActionPublisher: 'User Action: User 'operator01' performed 'DELETE_CRITICAL_RESOURCE' on 'db_master_table'.' with Topics: ['USER_ACTION', 'DELETE_CRITICAL_RESOURCE', 'CRITICAL_USER_ACTION', 'ALERT', 'SECURITY_ALERT']

--- Subscriber Router Node (Matching Subscribers to Message) ---
Message Topics: ['USER_ACTION', 'DELETE_CRITICAL_RESOURCE', 'CRITICAL_USER_ACTION', 'ALERT', 'SECURITY_ALERT']
Subscribers Matched for these topics: ['subscriber_alpha', 'subscriber_beta', 'subscriber_gamma', 'subscriber_audit']
Conditional Subscriber Dispatch: Routing to next pending subscriber: subscriber_alpha

--- SUBSCRIBER_ALPHA Node ---
SUBSCRIBER_ALPHA: Processed message

In [57]:
event_system_maintenance = {
    "source_type": "SYSTEM_INTERNAL_EVENT",
    "details": {"event_name": "SCHEDULED_MAINTENANCE_START", "component": "API_GATEWAY", "severity": "INFO"}
}
run_event_through_pubsub(event_system_maintenance) # Expect Gamma



<<<<< STARTING NEW EVENT: ID 5393cdc4-b982-4417-9c88-473e42cbe954, Source Type: SYSTEM_INTERNAL_EVENT >>>>>

--- Initial Event Intake ---

--- Event Source Router (Selecting Publisher) ---
Routing to SYSTEM event publisher.

--- Publisher: SystemEventPublisher (Event ID: 5393cdc4-b982-4417-9c88-473e42cbe954) ---
Publishing from SystemEventPublisher: 'System Event: 'SCHEDULED_MAINTENANCE_START' from component 'API_GATEWAY', Severity: INFO.' with Topics: ['SYSTEM_EVENT', 'SYSTEM_API_GATEWAY', 'SEVERITY_INFO', 'MAINTENANCE']

--- Subscriber Router Node (Matching Subscribers to Message) ---
Message Topics: ['SYSTEM_EVENT', 'SYSTEM_API_GATEWAY', 'SEVERITY_INFO', 'MAINTENANCE']
Subscribers Matched for these topics: ['subscriber_gamma']
Conditional Subscriber Dispatch: Routing to next pending subscriber: subscriber_gamma

--- SUBSCRIBER_GAMMA Node ---
SUBSCRIBER_GAMMA: Processed message 'System Event: 'SCHEDULED_MAINTENANCE_START' from component 'API_GATEWAY', Severity: INFO.' (Originated f

In [58]:
event_sensor_normal = {
    "source_type": "SENSOR_EVENT",
    "details": {"sensor_id": "flow_H2O", "reading": 10.2, "unit": "L/min", "status": "NORMAL"}
}
run_event_through_pubsub(event_sensor_normal) # May not trigger high-alert subscribers directly



<<<<< STARTING NEW EVENT: ID 9edd91f4-c04d-48d1-9848-8f5d03c27e22, Source Type: SENSOR_EVENT >>>>>

--- Initial Event Intake ---

--- Event Source Router (Selecting Publisher) ---
Routing to SENSOR event publisher.

--- Publisher: SensorPublisher (Event ID: 9edd91f4-c04d-48d1-9848-8f5d03c27e22) ---
Publishing from SensorPublisher: 'Sensor Event: ID flow_H2O, Reading 10.2L/min, Status: NORMAL.' with Topics: ['SENSOR_DATA']

--- Subscriber Router Node (Matching Subscribers to Message) ---
Message Topics: ['SENSOR_DATA']
Subscribers Matched for these topics: []
Conditional Subscriber Dispatch: No subscribers matched. Routing to summary.

--- Event Processing Summary ---
Event ID: 9edd91f4-c04d-48d1-9848-8f5d03c27e22
Processed by Publisher: SensorPublisher
Published Message: Sensor Event: ID flow_H2O, Reading 10.2L/min, Status: NORMAL.
Message Topics: ['SENSOR_DATA']
Matched Subscribers: []
Completed Subscribers: set()
Subscriber Logs for this event:
---------------------------------
<<<

In [59]:
event_user_update = {
    "source_type": "USER_ACTION_EVENT",
    "details": {"user_id": "jane.doe", "action": "UPDATE_PROFILE", "resource": "user_profile_jane.doe"}
}
run_event_through_pubsub(event_user_update) # Expect Beta, Audit



<<<<< STARTING NEW EVENT: ID 16f03159-db30-473e-845c-dc6dfc61691c, Source Type: USER_ACTION_EVENT >>>>>

--- Initial Event Intake ---

--- Event Source Router (Selecting Publisher) ---
Routing to USER_ACTION event publisher.

--- Publisher: UserActionPublisher (Event ID: 16f03159-db30-473e-845c-dc6dfc61691c) ---
Publishing from UserActionPublisher: 'User Action: User 'jane.doe' performed 'UPDATE_PROFILE' on 'user_profile_jane.doe'.' with Topics: ['USER_ACTION', 'UPDATE_PROFILE']

--- Subscriber Router Node (Matching Subscribers to Message) ---
Message Topics: ['USER_ACTION', 'UPDATE_PROFILE']
Subscribers Matched for these topics: ['subscriber_beta', 'subscriber_audit']
Conditional Subscriber Dispatch: Routing to next pending subscriber: subscriber_beta

--- SUBSCRIBER_BETA Node ---
SUBSCRIBER_BETA: Processed message 'User Action: User 'jane.doe' performed 'UPDATE_PROFILE' on 'user_profile_jane.doe'.' (Originated from: UserActionPublisher, Topics: ['USER_ACTION', 'UPDATE_PROFILE'])

-

# v4 + multiple input messages

The Pub-Sub is never about a single message. It is always about **a lot of messages**, right?

Okay, let's refactor the Pub-Sub system to handle a batch of messages. The core idea will be to introduce a loop within the LangGraph execution flow. The graph will take a list of initial event payloads, process them one by one through the existing publisher/subscriber logic, and aggregate the results.

**Key Changes:**

1.  **State Modification (`BatchPubSubState`):**
    *   `initial_batch_of_events`: A list of the raw event payloads to be processed.
    *   `current_event_index`: To keep track of which event in the batch is currently being processed.
    *   `current_event_id`, `current_event_payload`: To hold the data for the single event currently flowing through the individual pub-sub logic.
    *   `processed_event_summaries`: A list to store a summary/result from each event processed in the batch.
    *   The existing fields (like `publisher_identity`, `published_message_content`, `subscriber_logs`) will be reused for each event in the batch.

2.  **New Batch Control Nodes:**
    *   `batch_initializer_node`: Sets up the batch processing (e.g., `current_event_index = 0`).
    *   `select_next_event_from_batch_node`: Picks the event at `current_event_index` from the batch, populates `current_event_id` and `current_event_payload`, and **crucially resets the per-event state fields** (like `published_message_content`, `subscribers_matched_for_event`, `subscriber_logs`) to ensure a clean slate for each event.
    *   `aggregate_event_result_node`: After an event is fully processed by its subscribers, this node will take the relevant information from the state (e.g., the final state of `subscriber_logs` for that event, `published_message_content`) and append it to `processed_event_summaries`. It then increments `current_event_index`.
    *   `batch_finalization_node`: A terminal node that can print a summary of the entire batch processing.

3.  **Conditional Routing for the Batch Loop:**
    *   After `aggregate_event_result_node`, a conditional edge will check if `current_event_index < len(initial_batch_of_events)`.
        *   If true, it routes back to `select_next_event_from_batch_node`.
        *   If false, it routes to `batch_finalization_node` (or `END`).

4.  **Adaptation of Existing Nodes:**
    *   Publisher nodes will now use `state["current_event_payload"]` to get the details for the event they are publishing.
    *   The `reset_publication_state_fields()` logic is now centralized in `select_next_event_from_batch_node`.



This version introduces an internal loop to process a batch of messages. Each message in the batch goes through its own publication and subscriber matching cycle. The results are collected, and the graph terminates after all messages in the input batch have been processed. This makes the LangGraph system itself responsible for iterating through the "simultaneously published" messages.

In [71]:
# pub_sub_langgraph_v4_batch_processing_FIXED.py

from typing import TypedDict, List, Optional, Annotated, Set, Dict, Any
import operator
import uuid

from langgraph.graph import StateGraph, END

# --- 1. Define Subscriber Interests (remains the same) ---
SUBSCRIBER_INTERESTS = {
    "subscriber_alpha": ["ALERT", "CRITICAL_SENSOR"],
    "subscriber_beta": ["UPDATE", "USER_ACTION", "GENERAL"],
    "subscriber_gamma": ["ALERT", "SYSTEM_HEALTH", "STATS", "MAINTENANCE"],
    "subscriber_audit": ["USER_ACTION", "CRITICAL_SENSOR", "SYSTEM_CONFIG_CHANGE", "SECURITY_ALERT"]
}

# --- 2. Define the State for Batch Processing (remains the same) ---
class EventSummary(TypedDict):
    event_id: str
    original_payload: Dict[str, Any]
    publisher_identity: Optional[str]
    published_message_content: Optional[str]
    published_message_topics: List[str]
    matched_subscribers: List[str]
    completed_subscribers: Set[str]
    subscriber_logs: Dict[str, List[str]]

class BatchPubSubState(TypedDict):
    initial_batch_of_events: List[Dict[str, Any]]
    current_event_index: int
    current_event_id: Optional[str]
    current_event_payload: Optional[Dict[str, Any]]
    publisher_identity: Optional[str]
    published_message_content: Optional[str]
    published_message_topics: List[str]
    subscribers_matched_for_event: List[str]
    subscribers_completed_for_event: Set[str]
    subscriber_logs: Annotated[Dict[str, List[str]], operator.ior]
    processed_event_summaries: Annotated[List[EventSummary], operator.add]

# --- Helper to initialize/reset logs for an event (remains the same) ---
def get_initial_subscriber_logs_for_event() -> Dict[str, List[str]]:
    return {sub_name: [] for sub_name in SUBSCRIBER_INTERESTS.keys()}

# --- Utility to reset state fields FOR A SINGLE EVENT within the batch (remains the same) ---
def reset_fields_for_new_event_in_batch() -> Dict:
    return {
        "publisher_identity": None,
        "published_message_content": None,
        "published_message_topics": [],
        "subscribers_matched_for_event": [],
        "subscribers_completed_for_event": set(),
        "subscriber_logs": get_initial_subscriber_logs_for_event(),
        "current_event_id": None, # Ensure these are reset too
        "current_event_payload": None
    }

# --- 3. Batch Control Nodes ---
# MODIFIED batch_initializer_node
def batch_initializer_node(state: BatchPubSubState) -> Dict:
    print("\n--- Batch Initializer Node ---")
    batch_len = len(state.get("initial_batch_of_events", []))
    print(f"Initializing batch processing for {batch_len} events.")
    return {
        "current_event_index": 0,
        "processed_event_summaries": []
    }

# MODIFIED select_next_event_from_batch_node
def select_next_event_from_batch_node(state: BatchPubSubState) -> Dict:
    print("\n--- Selecting Next Event from Batch Node ---")
    idx = state["current_event_index"]
    batch = state.get("initial_batch_of_events", []) # Default to empty list
    
    # Always reset per-event fields. If no event is selected, they remain reset.
    update_dict = reset_fields_for_new_event_in_batch()

    if idx >= len(batch):
        print(f"select_next_event: Index {idx} is out of bounds for batch size {len(batch)}. No event selected.")
        # current_event_payload remains None from reset_fields...
        return update_dict 
    
    current_event_raw_payload = batch[idx]
    event_id = current_event_raw_payload.get("event_id", str(uuid.uuid4()))
    
    print(f"Processing event {idx + 1}/{len(batch)}: ID {event_id}, Payload: {current_event_raw_payload}")
    
    update_dict.update({
        "current_event_id": event_id,
        "current_event_payload": current_event_raw_payload,
    })
    return update_dict

# aggregate_event_result_node (remains mostly the same, minor safety for Nones)
def aggregate_event_result_node(state: BatchPubSubState) -> Dict:
    print("\n--- Aggregating Event Result Node ---")
    
    # This node is now only called if an event was actually processed.
    # However, good practice to use .get with defaults.
    event_summary: EventSummary = {
        "event_id": state.get("current_event_id", "N/A_Error"),
        "original_payload": state.get("current_event_payload", {}), # Should be set if we got here via event processing
        "publisher_identity": state.get("publisher_identity"),
        "published_message_content": state.get("published_message_content"),
        "published_message_topics": list(state.get("published_message_topics", [])),
        "matched_subscribers": list(state.get("subscribers_matched_for_event", [])),
        "completed_subscribers": set(state.get("subscribers_completed_for_event", set())),
        "subscriber_logs": dict(state.get("subscriber_logs", {}))
    }
    
    next_index = state["current_event_index"] + 1 # Increment index for the *next* potential event
    print(f"Finished processing event ID {event_summary['event_id']}. Aggregated its summary. Next index to try: {next_index}.")
    
    return {
        "processed_event_summaries": [event_summary],
        "current_event_index": next_index
    }

# batch_finalization_node (remains the same)
def batch_finalization_node(state: BatchPubSubState) -> Dict:
    print("\n--- Batch Finalization Node ---")
    summaries = state.get("processed_event_summaries", [])
    print(f"Batch processing complete. Total events processed: {len(summaries)}")
    for i, summary in enumerate(summaries):
        print(f"\n  Summary for Batch Event {i+1} (Original ID: {summary['event_id']}):")
        print(f"    Published by: {summary['publisher_identity']}")
        print(f"    Message: '{summary['published_message_content']}'")
        print(f"    Topics: {summary['published_message_topics']}")
        print(f"    Matched Subscribers: {summary['matched_subscribers']}")
    print("--------------------------------------")
    return {}

# --- 4. Specialized Publisher Nodes (remain the same) ---
def publisher_sensor_events_node(state: BatchPubSubState) -> Dict: # ... (no changes)
    event_id = state["current_event_id"] 
    payload_details = state["current_event_payload"].get("details", {}) 
    publisher_id = "SensorPublisher"
    print(f"\n--- Publisher: {publisher_id} (Current Event ID: {event_id}) ---")
    reading = payload_details.get("reading", "N/A")
    unit = payload_details.get("unit", "")
    status = payload_details.get("status", "NORMAL").upper()
    message_content = f"Sensor Event: ID {payload_details.get('sensor_id', 'UnknownSensor')}, Reading {reading}{unit}, Status: {status}."
    derived_topics = ["SENSOR_DATA"]
    if status == "CRITICAL":
        derived_topics.extend(["CRITICAL_SENSOR", "ALERT"])
    elif status == "WARNING":
        derived_topics.extend(["WARNING_SENSOR", "ALERT"])
    print(f"Publishing from {publisher_id} for Event ID {event_id}: '{message_content}' with Topics: {derived_topics}")
    return { 
        "publisher_identity": publisher_id,
        "published_message_content": message_content,
        "published_message_topics": derived_topics,
    }
def publisher_user_actions_node(state: BatchPubSubState) -> Dict: # ... (no changes)
    event_id = state["current_event_id"]
    payload_details = state["current_event_payload"].get("details", {})
    publisher_id = "UserActionPublisher"
    print(f"\n--- Publisher: {publisher_id} (Current Event ID: {event_id}) ---")
    user_id = payload_details.get("user_id", "anonymous")
    action = payload_details.get("action", "unknown_action").upper()
    resource = payload_details.get("resource", "N/A")
    message_content = f"User Action: User '{user_id}' performed '{action}' on '{resource}'."
    derived_topics = ["USER_ACTION", action]
    if "DELETE" in action or "SUSPEND" in action:
        derived_topics.extend(["CRITICAL_USER_ACTION", "ALERT", "SECURITY_ALERT"])
    if action == "CONFIG_CHANGE" and resource.startswith("system"):
        derived_topics.append("SYSTEM_CONFIG_CHANGE")
    print(f"Publishing from {publisher_id} for Event ID {event_id}: '{message_content}' with Topics: {derived_topics}")
    return {
        "publisher_identity": publisher_id,
        "published_message_content": message_content,
        "published_message_topics": derived_topics,
    }
def publisher_system_events_node(state: BatchPubSubState) -> Dict: # ... (no changes)
    event_id = state["current_event_id"]
    payload_details = state["current_event_payload"].get("details", {})
    publisher_id = "SystemEventPublisher"
    print(f"\n--- Publisher: {publisher_id} (Current Event ID: {event_id}) ---")
    event_name = payload_details.get("event_name", "generic_system_event").upper()
    severity = payload_details.get("severity", "INFO").upper()
    component = payload_details.get("component", "CORE").upper()
    message_content = f"System Event: '{event_name}' from component '{component}', Severity: {severity}."
    derived_topics = ["SYSTEM_EVENT", f"SYSTEM_{component}", f"SEVERITY_{severity}"]
    if severity in ["ERROR", "FATAL", "CRITICAL"]:
        derived_topics.append("ALERT")
    if event_name == "HEALTH_CHECK_FAIL":
        derived_topics.append("SYSTEM_HEALTH")
    if "MAINTENANCE" in event_name:
        derived_topics.append("MAINTENANCE")
    if "STATS_REPORT" in event_name:
        derived_topics.append("STATS")
    print(f"Publishing from {publisher_id} for Event ID {event_id}: '{message_content}' with Topics: {derived_topics}")
    return {
        "publisher_identity": publisher_id,
        "published_message_content": message_content,
        "published_message_topics": derived_topics,
    }

# --- 5. Publisher Router Node (route_to_correct_publisher function remains the same) ---
def route_to_correct_publisher(state: BatchPubSubState) -> str: # ... (no changes)
    print("\n--- Event Source Router (Selecting Publisher for current event in batch) ---")
    source_type = state["current_event_payload"].get("source_type", "UNKNOWN").upper()
    event_id = state["current_event_id"]
    print(f"Routing Event ID {event_id} (source: {source_type}) to appropriate publisher.")
    if source_type == "SENSOR_EVENT":
        return "publisher_sensor"
    elif source_type == "USER_ACTION_EVENT":
        return "publisher_user_action"
    elif source_type == "SYSTEM_INTERNAL_EVENT":
        return "publisher_system"
    else:
        print(f"Warning: Unknown source_type '{source_type}' for Event ID {event_id}. Defaulting to system publisher.")
        return "publisher_system" 

# --- 6. Subscriber Logic (remain the same) ---
def subscriber_router_node(state: BatchPubSubState) -> Dict: # ... (no changes)
    event_id = state["current_event_id"]
    print(f"\n--- Subscriber Router Node (Current Event ID: {event_id}) ---")
    published_topics = state.get("published_message_topics", [])
    if not published_topics and not isinstance(published_topics, list):
        print(f"Error for Event ID {event_id}: No message topics found or not a list.")
        return {"subscribers_matched_for_event": []}
    matched_subscribers = []
    for sub_name, interests in SUBSCRIBER_INTERESTS.items():
        if any(topic in published_topics for topic in interests):
            matched_subscribers.append(sub_name)
    print(f"Event ID {event_id} - Message Topics: {published_topics}, Matched Subscribers: {matched_subscribers}")
    return {"subscribers_matched_for_event": matched_subscribers}
def create_subscriber_node(subscriber_name: str): # ... (no changes)
    def subscriber_node_logic(state: BatchPubSubState) -> Dict:
        event_id = state['current_event_id']
        print(f"\n--- {subscriber_name.upper()} Node (Current Event ID: {event_id}) ---")
        message = state["published_message_content"]
        topics = state["published_message_topics"]
        log_entry = (f"{subscriber_name.upper()}: Processed message '{message}' "
                     f"(Originated from: {state.get('publisher_identity', 'Unknown Publisher')}, Topics: {topics})")
        print(log_entry)
        new_log_update_for_this_event = {
            subscriber_name: [log_entry] 
        }
        updated_completed_set = set(state.get("subscribers_completed_for_event", set()))
        updated_completed_set.add(subscriber_name)
        return {
            "subscriber_logs": new_log_update_for_this_event,
            "subscribers_completed_for_event": updated_completed_set
        }
    return subscriber_node_logic
subscriber_alpha_node = create_subscriber_node("subscriber_alpha")
subscriber_beta_node = create_subscriber_node("subscriber_beta")
subscriber_gamma_node = create_subscriber_node("subscriber_gamma")
subscriber_audit_node = create_subscriber_node("subscriber_audit")

# --- 7. Conditional Logic for Batch and Subscriber Dispatch ---
# route_after_event_aggregation (remains the same)
def route_after_event_aggregation(state: BatchPubSubState) -> str: # ... (no changes)
    idx = state["current_event_index"] 
    batch_size = len(state.get("initial_batch_of_events", []))
    if idx < batch_size:
        print(f"Conditional Batch Route: More events in batch (next index {idx}). Routing to select_next_event.")
        return "select_next_event"
    else:
        print(f"Conditional Batch Route: All {batch_size} events processed. Routing to batch_finalize.")
        return "batch_finalize"
# route_to_next_subscriber_or_aggregate (remains the same)
def route_to_next_subscriber_or_aggregate(state: BatchPubSubState) -> str: # ... (no changes)
    event_id = state['current_event_id']
    matched_subscribers = state.get("subscribers_matched_for_event", [])
    completed_subscribers = state.get("subscribers_completed_for_event", set())
    if not matched_subscribers:
        print(f"Conditional Subscriber Dispatch (Event ID {event_id}): No subscribers matched. Routing to aggregate_result.")
        return "aggregate_result"
    for sub_name in matched_subscribers:
        if sub_name not in completed_subscribers:
            print(f"Conditional Subscriber Dispatch (Event ID {event_id}): Routing to subscriber: {sub_name}")
            return sub_name  
    print(f"Conditional Subscriber Dispatch (Event ID {event_id}): All matched subscribers processed. Routing to aggregate_result.")
    return "aggregate_result"

# NEW Conditional function after event selection
def check_if_event_selected_for_processing(state: BatchPubSubState) -> str:
    if state.get("current_event_payload") is not None:
        # An event was successfully selected and its payload is in the state
        print("check_if_event_selected: Event selected. Routing to publisher_router_hub.")
        return "publisher_router_hub"
    else:
        # No event was selected (empty batch from start, or end of batch reached by select_next_event)
        print("check_if_event_selected: No event selected. Batch processing will be finalized.")
        return "batch_finalize"

# --- 8. Define the Graph Workflow ---
workflow = StateGraph(BatchPubSubState)

# Batch control nodes
workflow.add_node("batch_initialize", batch_initializer_node)
workflow.add_node("select_next_event", select_next_event_from_batch_node)
workflow.add_node("aggregate_result", aggregate_event_result_node)
workflow.add_node("batch_finalize", batch_finalization_node)

# NEW Publisher Router Hub node
def publisher_router_hub_node(state: BatchPubSubState) -> Dict:
    print("\n--- Publisher Router Hub (Triggering publisher selection) ---")
    return {} # No state change, just a routing point
workflow.add_node("publisher_router_hub", publisher_router_hub_node)

# Publisher nodes (specialized ones)
workflow.add_node("publisher_sensor", publisher_sensor_events_node)
workflow.add_node("publisher_user_action", publisher_user_actions_node)
workflow.add_node("publisher_system", publisher_system_events_node)

# Subscriber-side nodes
workflow.add_node("subscriber_router", subscriber_router_node)
workflow.add_node("subscriber_alpha", subscriber_alpha_node)
workflow.add_node("subscriber_beta", subscriber_beta_node)
workflow.add_node("subscriber_gamma", subscriber_gamma_node)
workflow.add_node("subscriber_audit", subscriber_audit_node)

# --- Define Edges and Control Flow ---
workflow.set_entry_point("batch_initialize")
workflow.add_edge("batch_initialize", "select_next_event")

# MODIFIED: Conditional edge after selecting an event
workflow.add_conditional_edges(
    "select_next_event",
    check_if_event_selected_for_processing, # NEW conditional function
    {
        "publisher_router_hub": "publisher_router_hub", # If event selected, go to hub
        "batch_finalize": "batch_finalize"            # If no event, finalize
    }
)

# MODIFIED: Conditional edge FROM the new hub to correct publisher
workflow.add_conditional_edges(
    "publisher_router_hub", # Source is now the hub
    route_to_correct_publisher, # Existing function
    {
        "publisher_sensor": "publisher_sensor",
        "publisher_user_action": "publisher_user_action",
        "publisher_system": "publisher_system",
    }
)

# All specialized publishers route to the common subscriber_router (same)
workflow.add_edge("publisher_sensor", "subscriber_router")
workflow.add_edge("publisher_user_action", "subscriber_router")
workflow.add_edge("publisher_system", "subscriber_router")

# Subscriber router dispatches (same)
workflow.add_conditional_edges(
    "subscriber_router",
    route_to_next_subscriber_or_aggregate,
    {
        "subscriber_alpha": "subscriber_alpha",
        "subscriber_beta": "subscriber_beta",
        "subscriber_gamma": "subscriber_gamma",
        "subscriber_audit": "subscriber_audit",
        "aggregate_result": "aggregate_result" 
    }
)

# After each subscriber finishes, loop back (same)
workflow.add_edge("subscriber_alpha", "subscriber_router")
workflow.add_edge("subscriber_beta", "subscriber_router")
workflow.add_edge("subscriber_gamma", "subscriber_router")
workflow.add_edge("subscriber_audit", "subscriber_router")

# After an event's result is aggregated, decide if more events in batch or finalize (same)
workflow.add_conditional_edges(
    "aggregate_result",
    route_after_event_aggregation,
    {
        "select_next_event": "select_next_event", 
        "batch_finalize": "batch_finalize"       
    }
)

workflow.add_edge("batch_finalize", END)

# --- 9. Compile the Graph ---
app = workflow.compile()

# --- 10. Run the Graph with a Batch of Events (same) ---
def run_batch_through_pubsub(batch_of_event_payloads: List[Dict[str, Any]]):
    print(f"\n\n<<<<< STARTING NEW BATCH OF {len(batch_of_event_payloads)} EVENTS >>>>>")
    initial_state_for_batch = { "initial_batch_of_events": batch_of_event_payloads }
    final_graph_state = None
    # Added recursion_limit for potentially deep loops with many events/subscribers per event
    for step_output in app.stream(initial_state_for_batch, {"recursion_limit": 200}):
        node_name = list(step_output.keys())[0]
        final_graph_state = step_output[node_name] 
    print("\n--- Final State After Batch Processing (from last node output) ---")
    if final_graph_state:
        print(f"Total unique events processed and summarized: {len(final_graph_state.get('processed_event_summaries', []))}")
    else:
        print("No final state captured.")
    print(f"<<<<< FINISHED BATCH PROCESSING >>>>>")

# # Example Batch of Events:
# batch_events = [
#     { "event_id": "event_001_sensor_critical", "source_type": "SENSOR_EVENT", "details": {"sensor_id": "temp_CPU0", "reading": 95.5, "unit": "C", "status": "CRITICAL"}},
#     { "source_type": "USER_ACTION_EVENT", "details": {"user_id": "operator01", "action": "DELETE_CRITICAL_RESOURCE", "resource": "db_master_table"}},
#     { "event_id": "event_003_system_maint", "source_type": "SYSTEM_INTERNAL_EVENT", "details": {"event_name": "SCHEDULED_MAINTENANCE_START", "component": "API_GATEWAY", "severity": "INFO"}},
# ]
# run_batch_through_pubsub(batch_events)

# # Test with an empty batch
# print("\n\n--- TESTING WITH EMPTY BATCH ---")
# run_batch_through_pubsub([])

# # Test with a single event batch
# print("\n\n--- TESTING WITH SINGLE EVENT BATCH ---")
# run_batch_through_pubsub([
#     {"source_type": "SYSTEM_INTERNAL_EVENT", "details": {"event_name": "STATS_REPORT", "component": "ANALYTICS", "severity": "INFO"}}
# ])

In [72]:
# Test with an empty batch
print("\n\n--- TESTING WITH EMPTY BATCH ---")
run_batch_through_pubsub([])





--- TESTING WITH EMPTY BATCH ---


<<<<< STARTING NEW BATCH OF 0 EVENTS >>>>>

--- Batch Initializer Node ---
Initializing batch processing for 0 events.

--- Selecting Next Event from Batch Node ---
select_next_event: Index 0 is out of bounds for batch size 0. No event selected.
check_if_event_selected: No event selected. Batch processing will be finalized.

--- Batch Finalization Node ---
Batch processing complete. Total events processed: 0
--------------------------------------

--- Final State After Batch Processing (from last node output) ---
No final state captured.
<<<<< FINISHED BATCH PROCESSING >>>>>


In [73]:
# Test with a single event batch
print("\n\n--- TESTING WITH SINGLE EVENT BATCH ---")
run_batch_through_pubsub([
{
"source_type": "SYSTEM_INTERNAL_EVENT",
"details": {"event_name": "STATS_REPORT", "component": "ANALYTICS", "severity": "INFO"}
}
])



--- TESTING WITH SINGLE EVENT BATCH ---


<<<<< STARTING NEW BATCH OF 1 EVENTS >>>>>

--- Batch Initializer Node ---
Initializing batch processing for 1 events.

--- Selecting Next Event from Batch Node ---
Processing event 1/1: ID c0cc6f5b-e57a-476d-bca6-a8c8eff9572b, Payload: {'source_type': 'SYSTEM_INTERNAL_EVENT', 'details': {'event_name': 'STATS_REPORT', 'component': 'ANALYTICS', 'severity': 'INFO'}}
check_if_event_selected: Event selected. Routing to publisher_router_hub.

--- Publisher Router Hub (Triggering publisher selection) ---

--- Event Source Router (Selecting Publisher for current event in batch) ---
Routing Event ID c0cc6f5b-e57a-476d-bca6-a8c8eff9572b (source: SYSTEM_INTERNAL_EVENT) to appropriate publisher.

--- Publisher: SystemEventPublisher (Current Event ID: c0cc6f5b-e57a-476d-bca6-a8c8eff9572b) ---
Publishing from SystemEventPublisher for Event ID c0cc6f5b-e57a-476d-bca6-a8c8eff9572b: 'System Event: 'STATS_REPORT' from component 'ANALYTICS', Severity: INFO.' w

In [74]:
# Example Batch of Events:
batch_events = [
    { 
        "event_id": "event_001_sensor_critical", # Optional: pre-assign IDs for traceability
        "source_type": "SENSOR_EVENT",
        "details": {"sensor_id": "temp_CPU0", "reading": 95.5, "unit": "C", "status": "CRITICAL"}
    },
    { 
        "source_type": "USER_ACTION_EVENT", # ID will be auto-generated
        "details": {"user_id": "operator01", "action": "DELETE_CRITICAL_RESOURCE", "resource": "db_master_table"}
    },
    { 
        "event_id": "event_003_system_maint",
        "source_type": "SYSTEM_INTERNAL_EVENT",
        "details": {"event_name": "SCHEDULED_MAINTENANCE_START", "component": "API_GATEWAY", "severity": "INFO"}
    },
    { 
        "source_type": "USER_ACTION_EVENT",
        "details": {"user_id": "jane.doe", "action": "UPDATE_PROFILE", "resource": "user_profile_jane.doe"}
    },
    { # An event that might not trigger many subscribers
        "source_type": "SENSOR_EVENT",
        "details": {"sensor_id": "humidity_room2", "reading": 45.0, "unit": "%", "status": "NORMAL"}
    }
]

run_batch_through_pubsub(batch_events)





<<<<< STARTING NEW BATCH OF 5 EVENTS >>>>>

--- Batch Initializer Node ---
Initializing batch processing for 5 events.

--- Selecting Next Event from Batch Node ---
Processing event 1/5: ID event_001_sensor_critical, Payload: {'event_id': 'event_001_sensor_critical', 'source_type': 'SENSOR_EVENT', 'details': {'sensor_id': 'temp_CPU0', 'reading': 95.5, 'unit': 'C', 'status': 'CRITICAL'}}
check_if_event_selected: Event selected. Routing to publisher_router_hub.

--- Publisher Router Hub (Triggering publisher selection) ---

--- Event Source Router (Selecting Publisher for current event in batch) ---
Routing Event ID event_001_sensor_critical (source: SENSOR_EVENT) to appropriate publisher.

--- Publisher: SensorPublisher (Current Event ID: event_001_sensor_critical) ---
Publishing from SensorPublisher for Event ID event_001_sensor_critical: 'Sensor Event: ID temp_CPU0, Reading 95.5C, Status: CRITICAL.' with Topics: ['SENSOR_DATA', 'CRITICAL_SENSOR', 'ALERT']

--- Subscriber Router Node